# Census Reporter API Query

In [3]:
import requests
import urllib
import json
import pandas as pd

def get_table_data(table_ids):
    api_url = 'https://api.censusreporter.org/1.0/data/show/latest?'
    params = {'table_ids':','.join(table_ids),
             'geo_ids':'16000US3651000,860|16000US3651000',
              'primary_geo_id':'16000US3651000'}
    params_enc = urllib.urlencode(params)
    data = json.loads(requests.get(api_url + params_enc).text)
    return data

def get_table_as_json(table_ids):
    api_url = 'https://api.censusreporter.org/1.0/data/show/latest?'
    params = {'table_ids':','.join(table_ids),
             'geo_ids':'16000US3651000,860|16000US3651000',
              'primary_geo_id':'16000US3651000'}
    params_enc = urllib.urlencode(params)
    data = requests.get(api_url + params_enc).text
    return data

In [2]:
d = get_table_data(['B01001'])

In [4]:
d['data']['86000US07002']['B01001']['estimate']

NameError: name 'd' is not defined

In [5]:
# Adapted from https://gist.github.com/JoeGermuska/1ed425c068d540326854
def prep_for_pandas(json_data,include_moe=False):
    """Given a dict of dicts as they come from a Census Reporter API call, set it up to be amenable to pandas.DataFrame.from_dict"""
    result = {}
    for geoid, tables in json_data.iteritems():
        flat = {}
        for table, values in tables.iteritems():
            for kind, columns in values.iteritems():
                if kind == 'estimate':
                    flat.update(columns)
                elif kind == 'error' and include_moe:
                    renamed = dict((k+"_moe",v) for k,v in columns.iteritems())
                    flat.update(renamed)
        result[geoid] = flat
    return result

In [6]:
x = prep_for_pandas(d['data'])
x['86000US07002'] == d['data']['86000US07002']['B01001']['estimate']

NameError: name 'd' is not defined

In [7]:
pd.DataFrame(x)

NameError: name 'x' is not defined

In [8]:
def expand_column_names(col_dict):
    # Get the min and max indentation levels
    level_range = list(sorted(set(coldata['indent'] for colkey, coldata in col_dict.iteritems())))
    max_level, min_level = max(level_range), min(level_range)
    prev_level = min(level_range)
    curr_level = min(level_range)
    # loop through columns one at a time.
    # at each step, if we have increased the indent level,
    # add to the column prefix
    prefix = []
    out_names = {}
    for colkey in sorted(col_dict):
        coldata = col_dict[colkey]
        # print colkey, '=>', coldata['name']
        
        clean_name = coldata['name'].strip(':')
        
        if coldata['indent'] == min_level:
            prefix = [clean_name]
            out_names[colkey] =  ' '.join(prefix)
        elif coldata['indent'] > prev_level: #and coldata['indent'] != max_level:
            prefix.append(clean_name)
            out_names[colkey] = ' '.join(prefix)
        elif coldata['indent'] == prev_level:
            prefix.pop()
            prefix.append(clean_name)
            out_names[colkey] = ' '.join(prefix)
        elif coldata['indent'] < prev_level: # gone down a step
            prefix.pop() # remove the last item
            prefix.pop() # and the one before it
            prefix.append(clean_name)
            out_names[colkey] = ' '.join(prefix)
        else:
            out_names[colkey] = ' '.join(prefix + [clean_name])
        prev_level = coldata['indent']
    return out_names

In [10]:
# BUILD PANDAS DATAFRAME FROM CensusReporter TABLEID
def dataframe_from_json(table_name):
    d = get_table_data([table_name])
    df = pd.DataFrame.from_dict(prep_for_pandas(d['data']), orient='index')
    
    columns_in_order = list(sorted(df.columns))
    df = df[columns_in_order]

    columns_to_names = expand_column_names(d['tables'][table_name]['columns'])
    new_columns = [columns_to_names[colkey] for colkey in df.columns]

    df.columns = new_columns
    new_index = [rowname.split('US')[-1] for rowname in df.index]
    df.index = new_index
    df.index.name='ZIP/Loc Code'
    return df

# SAMPLE TABLES for NYC:
# B01001 - Demographics by Zip Code/Sex/Age
# B19013 - Household Income by Zip Code
# B25006 - Head of Household by Race/Ethnic Group
# EXAMPLE OF FUNCTION CALL => Insert specific Table ID (table_name) from CensusReporter

df = dataframe_from_json('B19013') # B01001 B25006
#df.head()

df.sort_values('Median household income in the past 12 months (in 2015 Inflation-adjusted dollars)', ascending=True)
#df.columns = zip(sorted(d['tables']['B01001']['columns'].keys()), expand_column_names(d['tables']['B01001']['columns']))

,Median household income in the past 12 months (in 2015 Inflation-adjusted dollars)
ZIP/Loc Code,
10454,20210.0
10453,23095.0
10460,23209.0
10455,23253.0
10459,23761.0
10456,23956.0
10457,24750.0
10452,24790.0
10451,26048.0


In [1]:
# Tests to make sure columns are lined up.
# May want to add some here.
#assert df['Total Female 15 to 17 years'].loc['3651000'] == d['data']['16000US3651000']['B01001']['estimate']['B01001030']

#assert df['Total Female'].loc['3651000'] == d['data']['16000US3651000']['B01001']['estimate']['B01001026']

In [25]:
columns_to_names = {col : d['tables']['B01001']['columns'][col]['name'] for col in d['tables']['B01001']['columns'].keys()}
for colkey in sorted(d['data']['86000US07036']['B01001']['estimate'].keys()):
    print columns_to_names[colkey], d['data']['86000US07036']['B01001']['estimate'][colkey]

Total: 42894.0
Male: 20771.0
Under 5 years 1316.0
5 to 9 years 1267.0
10 to 14 years 1068.0
15 to 17 years 860.0
18 and 19 years 715.0
20 years 346.0
21 years 232.0
22 to 24 years 858.0
25 to 29 years 1699.0
30 to 34 years 1428.0
35 to 39 years 1545.0
40 to 44 years 1553.0
45 to 49 years 1409.0
50 to 54 years 1595.0
55 to 59 years 1493.0
60 and 61 years 370.0
62 to 64 years 793.0
65 and 66 years 436.0
67 to 69 years 435.0
70 to 74 years 558.0
75 to 79 years 292.0
80 to 84 years 259.0
85 years and over 244.0
Female: 22123.0
Under 5 years 1227.0
5 to 9 years 1107.0
10 to 14 years 1037.0
15 to 17 years 868.0
18 and 19 years 571.0
20 years 339.0
21 years 295.0
22 to 24 years 810.0
25 to 29 years 1393.0
30 to 34 years 1767.0
35 to 39 years 1375.0
40 to 44 years 1473.0
45 to 49 years 1462.0
50 to 54 years 1840.0
55 to 59 years 1911.0
60 and 61 years 567.0
62 to 64 years 776.0
65 and 66 years 424.0
67 to 69 years 694.0
70 to 74 years 655.0
75 to 79 years 456.0
80 to 84 years 489.0
85 years an

In [40]:
d['data']['86000US07002']

{u'B01001': {u'error': {u'B01001001': 42.0,
   u'B01001002': 838.0,
   u'B01001003': 263.0,
   u'B01001004': 291.0,
   u'B01001005': 330.0,
   u'B01001006': 265.0,
   u'B01001007': 136.0,
   u'B01001008': 159.0,
   u'B01001009': 125.0,
   u'B01001010': 257.0,
   u'B01001011': 373.0,
   u'B01001012': 401.0,
   u'B01001013': 406.0,
   u'B01001014': 351.0,
   u'B01001015': 353.0,
   u'B01001016': 295.0,
   u'B01001017': 288.0,
   u'B01001018': 194.0,
   u'B01001019': 233.0,
   u'B01001020': 171.0,
   u'B01001021': 178.0,
   u'B01001022': 207.0,
   u'B01001023': 163.0,
   u'B01001024': 168.0,
   u'B01001025': 115.0,
   u'B01001026': 832.0,
   u'B01001027': 343.0,
   u'B01001028': 292.0,
   u'B01001029': 346.0,
   u'B01001030': 183.0,
   u'B01001031': 157.0,
   u'B01001032': 158.0,
   u'B01001033': 150.0,
   u'B01001034': 211.0,
   u'B01001035': 372.0,
   u'B01001036': 288.0,
   u'B01001037': 286.0,
   u'B01001038': 330.0,
   u'B01001039': 383.0,
   u'B01001040': 258.0,
   u'B01001041': 316

In [8]:
# QUERY TABLE DATA & LOAD IT INTO A PANDAS DATAFRAME 
tbl_id = 'B01001'

df = pd.DataFrame(columns=create_column_multiindex(d['tables']['B01001']['columns']))

rows = []
for location_code in d['data']:
    if len(location_code.split('US')[-1]) == 5:
        zipcode = location_code.split('US')[-1]
        row = {'zip':zipcode}
        male_idx = list(sorted(d['data'][location_code][tbl_id]['estimate'].keys()))
        
        for colkey in sorted(d['data'][location_code][tbl_id]['estimate'].keys()):
            
            row[columns_to_names[colkey]] = d['data'][location_code][tbl_id]['estimate'][colname]
            #print columns_to_names[colname], "\t", d['data'][location_code][tbl_id]['estimate'][colname]
        rows.append(row)

# NYC Age & Male/Female Demographics by zip code         
df = pd.DataFrame(rows).groupby('zip').sum()
df.columns = [df.columns[-1]] + list(df.columns[:-1])
#df = df.transpose()
#df.tail()

In [12]:
df.T.loc['07036']

Under 5 years         1068.0
10 to 14 years         860.0
15 to 17 years         715.0
18 and 19 years        339.0
20 years               232.0
21 years               858.0
22 to 24 years        1699.0
25 to 29 years        1428.0
30 to 34 years        1545.0
35 to 39 years        1473.0
40 to 44 years        1462.0
45 to 49 years        1267.0
5 to 9 years          1840.0
50 to 54 years        1911.0
55 to 59 years         567.0
60 and 61 years        776.0
62 to 64 years         436.0
65 and 66 years        694.0
67 to 69 years         558.0
70 to 74 years         292.0
75 to 79 years         489.0
80 to 84 years         587.0
85 years and over    22123.0
Female               20771.0
Male                 42894.0
Total                 1316.0
Name: 07036, dtype: float64

In [6]:
zip_codes = [x.split('US')[-1] for x in list((d['data']).iterkeys())]
print zip_codes

[u'10704', u'10312', u'10311', u'10310', u'11413', u'11412', u'07632', u'10314', u'11415', u'11419', u'11418', u'11414', u'11417', u'10455', u'07036', u'10069', u'11385', u'10465', u'10065', u'11096', u'10469', u'10468', u'11429', u'11364', u'10461', u'10460', u'10463', u'10462', u'3651000', u'10464', u'10467', u'10466', u'11422', u'11423', u'11420', u'11421', u'11426', u'11427', u'11424', u'07620', u'11428', u'11375', u'11367', u'07024', u'10018', u'10019', u'10153', u'10152', u'10154', u'10010', u'10011', u'10012', u'10013', u'10014', u'11355', u'10016', u'10017', u'11224', u'11225', u'11226', u'11220', u'11221', u'11222', u'11223', u'11228', u'11229', u'11020', u'11021', u'11218', u'11040', u'11042', u'10803', u'10705', u'10009', u'10003', u'10002', u'10001', u'10307', u'10007', u'10006', u'10005', u'10004', u'11237', u'11236', u'11235', u'11234', u'11233', u'11232', u'11231', u'11230', u'11430', u'11416', u'10128', u'10550', u'11239', u'11238', u'07020', u'11101', u'11219', u'11559